Cargar Librerias necesarias


In [39]:
pip install pandas pyarrow fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:00


In [9]:
from google.colab import files
import json
from google.colab import drive
import pandas as pd

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
ruta = "/content/transactions_50k.jsonl"
transacciones = pd.read_json(ruta, lines=True)
##transacciones.columns.tolist()
transacciones.head(3)

,id,created_at,updated_at,status,payment_method_type,amount_in_cents
0,1720000000-77942,2024-07-03 16:12:37,2024-07-03 16:13:29,APPROVED,"{'type': 'CARD', 'extra': {'bin': '450668', 'c...",5155064
1,1720000010-45698,2024-04-01 10:40:39,2024-04-01 10:43:49,APPROVED,"{'type': 'CARD', 'extra': {'bin': '512069', 'c...",14700093
2,1720000020-64182,2024-09-11 18:12:59,2024-09-11 18:14:23,APPROVED,"{'type': 'CARD', 'extra': {'bin': '438108', 'c...",2143363


Extraer el valor BIN de la columna payment_method_type:

In [23]:
transacciones["bin"] = transacciones["payment_method_type"].apply (
    lambda x: x.get("extra",{}).get("bin") if isinstance(x, dict) else None
)

Convertir fecha creacion a tipo date para extraer el día:

In [28]:
transacciones["created_at"] = pd.to_datetime(transacciones["created_at"])

Filtrar solo transacciones aprobadas:

In [29]:
transacciones_aprobadas = transacciones[transacciones["status"] == "APPROVED"].copy()

Crear columna de dia:

In [30]:
transacciones_aprobadas["day"] = transacciones_aprobadas["created_at"].dt.date

In [31]:
transacciones.head(3)

,id,created_at,updated_at,status,payment_method_type,amount_in_cents,bin
0,1720000000-77942,2024-07-03 16:12:37,2024-07-03 16:13:29,APPROVED,"{'type': 'CARD', 'extra': {'bin': '450668', 'c...",5155064,450668
1,1720000010-45698,2024-04-01 10:40:39,2024-04-01 10:43:49,APPROVED,"{'type': 'CARD', 'extra': {'bin': '512069', 'c...",14700093,512069
2,1720000020-64182,2024-09-11 18:12:59,2024-09-11 18:14:23,APPROVED,"{'type': 'CARD', 'extra': {'bin': '438108', 'c...",2143363,438108


Generación de vista agregada:

In [36]:
vista_transacciones = (
    transacciones_aprobadas.groupby(["bin","day"])
    .agg(
         cantidad_transacciones = ("id","count"),
         monto_total_aprobado = ("amount_in_cents","sum")
    )
    .reset_index()
)
print(vista_transacciones.head())

      bin         day  cantidad_transacciones  monto_total_aprobado
0  231030  2024-04-01                       1              28352749
1  231030  2024-04-02                       3              13007751
2  231030  2024-04-03                       3              26865765
3  231030  2024-04-04                       2              42546032
4  231030  2024-04-05                       1               9442522


Guardar Archivo en formato parquet:

In [38]:
vista_transacciones.to_parquet("/content/vista_transacciones.parquet" , index= False)